### Imports and configuration

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio

from rasterio.enums import Resampling

matplotlib.rc('figure', dpi=150)

binary_cmap = matplotlib.colors.ListedColormap(['green', 'blue'])
confusion_cmap = matplotlib.colors.ListedColormap(['gray', 'green', 'red', 'blue'])

### Helper functions

In [10]:
# makes inundation depth array binary
# 0 is not inundated and 1 is inundated
def make_binary(in_array):
    binary_array = in_array.copy().astype('uint8')
    binary_array = np.where(binary_array <= int(0), int(0), int(1))
    binary_array[binary_array==np.nan] = int(0)
    if isinstance(in_array, np.ma.MaskedArray):
        binary_array = np.ma.masked_array(binary_array, in_array.mask).astype('uint8')
    return binary_array.astype('uint8')

# takes two binary arrays where for each cell
# 0 is not inundated and 1 is inundated
def make_confusion_plot(truth_array, eval_array):
    confusion_classified = np.where(
        ((truth_array == 0) & (eval_array == 0)), 0, # TN = 0
        np.where(
            ((truth_array == 0) & (eval_array == 1)), 1, # FP = 1
            np.where(
                ((truth_array == 1) & (eval_array == 0)), 2, # FN = 2
                np.where(
                    ((truth_array == 1) & (eval_array == 1)), 3, # TP = 3
                    np.nan # else assign NaN
    ))))
    return confusion_classified

def make_confusion_stats(confusion_plot_array):
    _, counts = np.unique(confusion_plot_array, return_counts=True)

    tn = counts[0]
    fp = counts[1]
    fn = counts[2]
    tp = counts[3]

    prop_cor = (tp+tn)/(tn+fp+fn+tp)
    bias_ratio = (tp+fp)/(tp+fn)
    hit_rate = tp/(tp+fn)
    fitness = tp/(tp+fp+fn)

    print(
        f"{'True Negative (TN):':<20}{tn:>15}",
        f"\n{'False Positive (FP):':<20}{fp:>15}",
        f"\n{'False Negative (FN):':<20}{fn:>15}",
        f"\n{'True Postive (TP):':<20}{tp:>15}",
        "\n",
        f"\n{'Proportion Correct:':<20}{round(prop_cor,4):>15}",
        f"\n{'Bias Ratio:':<20}{round(bias_ratio,4):>15}",
        f"\n{'Hit Rate:':<20}{round(hit_rate,4):>15}",
        f"\n{'Fitness':<20}{round(fitness,4):>15}"
    )


def imshow_binary_array(in_array, save_path):
    fig, ax = plt.subplots()

    cax = ax.imshow(
        in_array,
        vmin = 0,
        vmax = 1,
        cmap = binary_cmap
        )

    cbar = fig.colorbar(
        cax,
        ticks=[0,.25,0.5,0.75,1],
        ax=ax,
        fraction=0.046,
        pad=0.03
        )

    cbar.ax.set_yticklabels(
        ['','Not inundated','','Inundated',''],
        rotation=90,
        va='center' # set vertical alignment
        )

    if save_path is not None:
        fig.tight_layout()
        plt.savefig(save_path, facecolor = 'w')
        
    plt.show()

def imshow_confusion(in_array, save_path):
    fig, ax = plt.subplots()

    cax = ax.imshow(
        in_array,
        vmin = 0,
        vmax = 3,
        cmap = confusion_cmap
        )

    cbar = fig.colorbar(
        cax,
        ticks=np.arange(0,4,0.375),
        ax=ax,
        fraction=0.046,
        pad=0.03
        )

    cbar.ax.set_yticklabels(
        ['', 'TN', '', 'FP', '', 'FN', '', 'TP', '','',''],
        rotation=90,
        va = 'center' # centered vertical alignment
        )

    if save_path is not None:
        fig.tight_layout()
        plt.savefig(save_path, facecolor = 'w')
        
    plt.show()

def make_confusion_stats_debug(confusion_plot_array):
    tp_is_zero = False

    _, counts = np.unique(confusion_plot_array, return_counts=True)

    tn = counts[0]
    fp = counts[1]
    fn = counts[2]

    if len(counts) < 4:
        tp = 0
        tp_is_zero = True

    prop_cor = (tp+tn)/(tn+fp+fn+tp)
    bias_ratio = (tp+fp)/(tp+fn)
    hit_rate = tp/(tp+fn)
    fitness = tp/(tp+fp+fn)

    print(
        f"{'True Negative (TN):':<20}{tn:>15}",
        f"\n{'False Positive (FP):':<20}{fp:>15}",
        f"\n{'False Negative (FN):':<20}{fn:>15}",
        f"\n{'True Postive (TP):':<20}{tp:>15}",
        "\n",
        f"\n{'Proportion Correct:':<20}{round(prop_cor,4):>15}",
        f"\n{'Bias Ratio:':<20}{round(bias_ratio,4):>15}",
        f"\n{'Hit Rate:':<20}{round(hit_rate,4):>15}",
        f"\n{'Fitness':<20}{round(fitness,4):>15}"
    )

    return counts, tp_is_zero

### Compound - Sep 14

In [3]:
compound_inun = rio.open('data_github/compound_inun_SEP14_alpha1.tif','r')
compound_inun_array = compound_inun.read(1,masked=True)
compound_inun_array_binary = make_binary(compound_inun_array)

SAR_inun = rio.open('data_github/SAR_SEP14_masked_projUTM18n.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        compound_inun_array_binary.shape[0],
        compound_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    compound_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_compound = make_confusion_plot(SAR_inun_array, compound_inun_array_binary)

make_confusion_stats(confusion_SAR_compound)

True Negative (TN):       148231495 
False Positive (FP):        8501014 
False Negative (FN):        2445921 
True Postive (TP):         12564667 
 
Proportion Correct:          0.9363 
Bias Ratio:                  1.4034 
Hit Rate:                    0.8371 
Fitness                      0.5344


### Inland - Sep 14

In [4]:
riv_inun = rio.open('data_github/030202_201809140600_depth_map_projUTM18n.tif','r')
riv_inun_array = riv_inun.read(1,masked=True)
riv_inun_array_binary = make_binary(riv_inun_array)

SAR_inun = rio.open('data_github/SAR_SEP14_masked_projUTM18n.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        riv_inun_array_binary.shape[0],
        riv_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    riv_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_riv = make_confusion_plot(SAR_inun_array, riv_inun_array_binary)

make_confusion_stats(confusion_SAR_riv)

True Negative (TN):       155325853 
False Positive (FP):         866006 
False Negative (FN):       14295730 
True Postive (TP):           527720 
 
Proportion Correct:          0.9113 
Bias Ratio:                   0.094 
Hit Rate:                    0.0356 
Fitness                      0.0336


### Coastal - Sep 14

In [5]:
surge_inun = rio.open('data_github/florence_surge_nhc_max_internal_mask.tif','r')
surge_inun_array = surge_inun.read(1,masked=True)
surge_inun_array_binary = make_binary(surge_inun_array)

SAR_inun = rio.open('data_github/SAR_SEP14_masked_projUTM18n.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        surge_inun_array_binary.shape[0],
        surge_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    surge_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_surge = make_confusion_plot(SAR_inun_array, surge_inun_array_binary)

make_confusion_stats(confusion_SAR_surge)

True Negative (TN):       148654132 
False Positive (FP):        8078377 
False Negative (FN):        2451487 
True Postive (TP):         12559101 
 
Proportion Correct:          0.9387 
Bias Ratio:                  1.3749 
Hit Rate:                    0.8367 
Fitness                      0.5439


### Compound - Sep 19

In [6]:
compound_inun = rio.open('data_github/compound_inun_SEP19_alpha1.tif','r')
compound_inun_array = compound_inun.read(1,masked=True)
compound_inun_array_binary = make_binary(compound_inun_array)

SAR_inun = rio.open('data_github/SAR_19SEP_clipped_proj_extr_msk.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        compound_inun_array_binary.shape[0],
        compound_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    compound_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_compound = make_confusion_plot(SAR_inun_array, compound_inun_array_binary)

make_confusion_stats(confusion_SAR_compound)

True Negative (TN):       128172843 
False Positive (FP):        3528146 
False Negative (FN):        7611399 
True Postive (TP):           206317 
 
Proportion Correct:          0.9202 
Bias Ratio:                  0.4777 
Hit Rate:                    0.0264 
Fitness                      0.0182


### Inland - Sep 19

In [7]:
riv_inun = rio.open('data_github/030202_201809192100_depth_map_projUTM18n.tif','r')
riv_inun_array = riv_inun.read(1,masked=True)
riv_inun_array_binary = make_binary(riv_inun_array)

SAR_inun = rio.open('data_github/SAR_19SEP_clipped_proj_extr_msk.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        riv_inun_array_binary.shape[0],
        riv_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    riv_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_riv = make_confusion_plot(SAR_inun_array, riv_inun_array_binary)

make_confusion_stats(confusion_SAR_riv)

True Negative (TN):       161706316 
False Positive (FP):        3528494 
False Negative (FN):        9590943 
True Postive (TP):           205973 
 
Proportion Correct:           0.925 
Bias Ratio:                  0.3812 
Hit Rate:                     0.021 
Fitness                      0.0155


### Coastal - Sep 19

In [11]:
surge_inun = rio.open('data_github/inun_surge_201809192100.tif','r')
surge_inun_array = surge_inun.read(1,masked=True)
surge_inun_array_binary = make_binary(surge_inun_array)

SAR_inun = rio.open('data_github/SAR_19SEP_clipped_proj_extr_msk.tif','r')
# resample band 1 of SAR_inun to shape of compound_inun
# with nearest neighbor interpolation
SAR_inun_array = SAR_inun.read(
    1,
    masked=True,
    out_shape=(
        surge_inun_array_binary.shape[0],
        surge_inun_array_binary.shape[1]
    ),
    resampling=Resampling.nearest
)

# assert compound inundation array and SAR inundation array have same shape
assert np.equal(
    surge_inun_array_binary.shape,
    SAR_inun_array.shape
    ).all()

confusion_SAR_surge = make_confusion_plot(SAR_inun_array, surge_inun_array_binary)

counts, tp_is_zero = make_confusion_stats_debug(confusion_SAR_surge)

True Negative (TN):       131700989 
False Positive (FP):        7817716 
False Negative (FN):      426757731 
True Postive (TP):                0 
 
Proportion Correct:          0.2326 
Bias Ratio:                  0.0183 
Hit Rate:                       0.0 
Fitness                         0.0


In [12]:
counts, tp_is_zero

(array([131700989,   7817716, 426757731]), True)